In [1]:
import numpy as np
from tqdm import tqdm
from solver.qp import qp
import pandas as pd
import torch
import gzip
import pickle
import warnings
from generate_instances import generate_softmarginsvm, generate_markowitz_portfolio_optimization, generate_convex_function_to_data, generate_regressor_selection

### Soft margin Support Vector Machine

In [3]:
def surrogate_gen():
    NUM = 500 # Number of points
    DIM = 2 # Dimension of the points
    C = 10 # regularization parameter (hardness of the margin)
    
    
    # 2-D mean of ones
    M1 = np.ones((DIM,))
    # 2-D mean of threes
    M2 = 3 * np.ones((DIM,))
    # 2-D covariance of 0.3
    C1 = np.diag(0.3 * np.ones((DIM,)))
    # 2-D covariance of 0.2
    C2 = np.diag(0.2 * np.ones((DIM,)))

    # generate 50 points from gaussian 1
    x1 = np.random.multivariate_normal(M1, C1, NUM) #generate gaussian
    # labels
    y1 = np.ones((x1.shape[0],))
    # generate 50 points from gaussian 2
    x2 = np.random.multivariate_normal(M2, C2, NUM)
    y2 = -np.ones((x2.shape[0],))
    # join
    X = np.concatenate((x1, x2), axis = 0)
    y = np.concatenate((y1, y2), axis = 0)

    Q, q, G, h, A, b, S = generate_softmarginsvm(y, X, C)    
    return Q, q, G, h, A, b, S

### Markowitz portfolio optimization

In [7]:
def surrogate_gen():
    np.random.seed(1)
    n_obs = 252
    n_assets = 100
    artificial_returns = np.random.randn(n_obs, n_assets) + 0.05 # minimum expected return
    returns = pd.DataFrame(artificial_returns)
    pbar = np.array(returns.mean()).reshape(1, -1)
    r_min = pbar.mean() # minimum return threshold
    Q, q, G, h, A, b, S = generate_markowitz_portfolio_optimization(artificial_returns, r_min, pbar)
    return Q, q, G, h, A, b, S

### Fiting convex function to given data

In [11]:
def surrogate_gen():
    
    # Generate 'u' values (from -2 to 2) for the independent variable
    NUM = 100
    u_values = np.linspace(-2.0, 2.0, num=NUM).reshape(-1, 1)
    Q, q, G, h, A, b, S = generate_convex_function_to_data(u_values)
    return Q, q, G, h, A, b, S

### Regressor selection problem

In [13]:
def surrogate_gen():
    
    # Set dimensions
    m = 10  # number of rows for A
    n = 20  # number of columns for A
    nopts = 100 # range of alpha values
    alpha_index = np.random.randint(0, nopts) # Select an alpha value from the range nopts
    Q, q, G, h, A, b, S = generate_regressor_selection(alpha_index, m, n)
    
    return Q, q, G, h, A, b, S

### create

In [14]:
import os

# directory = f'../../../../work/log1/darius.weber/Quadratic_Programming_Datasets/raw'
directory = f'Quadratic_Programming_Datasets/raw'
os.makedirs(directory, exist_ok=True)
warnings.filterwarnings("error")

ips = []
pkg_idx = 3 #identifier index
success_cnt = 0
fail_cnt = 0
max_ipm_steps = 0


max_iter = 100
num = 10

for i in tqdm(range(max_iter)):
    Q, q, G, h, A, b, S = surrogate_gen()
    print(np.array(A).size)
    try:
        res = qp(Q, q, G, h, A, b, callback=lambda res: res)
        max_ipm_steps = max(max_ipm_steps, res['iterations'])
        print("status",res['status'])
    except Exception as e:
        fail_cnt += 1
        warnings.warn(f'Optimization failed with error: {str(e)}')
        continue
    else:
        if res['status'] == 'optimal': #np.isnan(res.fun)?
            ips.append((torch.from_numpy(np.array(Q)).to(torch.float), torch.from_numpy(np.array(q)).to(torch.float), torch.from_numpy(np.array(G)).to(torch.float), torch.from_numpy(np.array(h)).to(torch.float), torch.from_numpy(np.array(A)).to(torch.float), torch.from_numpy(np.array(b)).to(torch.float), torch.from_numpy(np.array(S)).to(torch.float), res))
            print(success_cnt)
            success_cnt += 1
            
    if len(ips) >= 1000 or success_cnt == num:
        print("success")
        ips = [ip + (max_ipm_steps,) for ip in ips]
        with gzip.open(f'{directory}/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            print(f'Package {pkg_idx} saved')
            pkg_idx += 1

        ips = []
    if success_cnt >= num:
        break
warnings.resetwarnings()

  4%|▍         | 4/100 [00:00<00:02, 36.06it/s]

0
     pcost       dcost       gap    pres   dres
 0: -1.0006e+01 -1.0659e+01  1e+02  1e+01  4e-01
 1: -8.5844e+00 -8.9184e+00  2e+01  1e+00  5e-02
 2: -6.5641e+00 -6.5215e+00  3e+00  2e-01  6e-03
 3: -5.9155e+00 -5.9794e+00  4e-01  2e-02  7e-04
 4: -5.9064e+00 -5.9168e+00  2e-02  1e-03  3e-05
 5: -5.9113e+00 -5.9122e+00  1e-03  2e-05  8e-07
 6: -5.9117e+00 -5.9117e+00  7e-05  3e-07  1e-08
 7: -5.9117e+00 -5.9117e+00  1e-06  3e-09  1e-10
Optimal solution found.
status optimal
0
0
     pcost       dcost       gap    pres   dres
 0: -6.7306e+00 -8.3174e+00  1e+02  1e+01  4e-01
 1: -6.2364e+00 -7.2419e+00  2e+01  1e+00  4e-02
 2: -4.8178e+00 -5.1547e+00  3e+00  2e-01  7e-03
 3: -4.2548e+00 -4.4072e+00  3e-01  2e-02  6e-04
 4: -4.3214e+00 -4.3433e+00  2e-02  2e-05  6e-07
 5: -4.3365e+00 -4.3374e+00  1e-03  2e-07  8e-09
 6: -4.3372e+00 -4.3372e+00  4e-05  4e-09  1e-10
 7: -4.3372e+00 -4.3372e+00  5e-07  4e-11  2e-12
Optimal solution found.
status optimal
1
0
     pcost       dcost       gap

  9%|▉         | 9/100 [00:00<00:03, 24.88it/s]

0
     pcost       dcost       gap    pres   dres
 0: -1.2422e+01 -2.4563e+01  1e+02  2e+00  4e-01
 1: -1.1859e+01 -2.2615e+01  2e+01  1e-01  2e-02
 2: -1.1856e+01 -1.4575e+01  3e+00  2e-02  2e-03
 3: -1.2474e+01 -1.3088e+01  6e-01  4e-04  5e-05
 4: -1.2645e+01 -1.2748e+01  1e-01  2e-16  3e-16
 5: -1.2669e+01 -1.2701e+01  3e-02  2e-16  2e-16
 6: -1.2678e+01 -1.2689e+01  1e-02  2e-16  2e-16
 7: -1.2682e+01 -1.2687e+01  5e-03  2e-16  3e-16
 8: -1.2684e+01 -1.2685e+01  1e-03  1e-16  4e-16
 9: -1.2684e+01 -1.2685e+01  5e-04  2e-16  1e-15
10: -1.2684e+01 -1.2684e+01  4e-05  2e-16  4e-16
11: -1.2684e+01 -1.2684e+01  5e-07  4e-16  3e-16
Optimal solution found.
status optimal
8
0
     pcost       dcost       gap    pres   dres
 0: -6.3410e+00 -1.1166e+01  1e+02  4e+00  4e-01
 1: -5.8492e+00 -1.0365e+01  1e+01  4e-01  4e-02
 2: -5.4667e+00 -7.3075e+00  2e+00  3e-02  3e-03
 3: -5.8501e+00 -6.3989e+00  6e-01  6e-03  5e-04
 4: -6.0942e+00 -6.2365e+00  1e-01  3e-04  3e-05
 5: -6.1674e+00 -6.1880e+0

In [ ]:
# Open and load the .pkl.gz file
with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{pkg_idx-1}.pkl.gz', 'rb') as f:
    data = pickle.load(f)

# Now `data` contains the contents of the pickle file
print(data)